In [ ]:
using DifferentialEquations
using Plots
using LinearAlgebra
using Statistics
using QuadGK
using ForwardDiff
using TaylorSeries
using Printf
using ProgressMeter
using Base.Threads


In [ ]:

# ========================================
# Some Helper Functions
# ========================================

function printMsg(msg)
    println("="^40)
    println(msg)
    println("="^40)
end

In [ ]:
IN_SCRIPT = false
if match(r"^In\[[0-9]*\]$", @__FILE__) != nothing
    printMsg("Running in Jupyter")
    IN_SCRIPT = false
else
    printMsg("Running in script")
    IN_SCRIPT = true
    ENV["GKSwstype"]="nul"
end

In [ ]:
# Parameters
# Degree of taylor expansion
const Order = 5



In [ ]:
const shift_V_0 = 1
const V_0(x) = (x-shift_V_0)^4/12 - 1/2*(x-shift_V_0)^2 +2/10 * (shift_V_0-4) + 1

"""
Get the Taylor expansion and the dynamic functions
accordign to \$f(x) = -\\frac{\\partial V_0(x)}{\\partial x}\$

Argument:
    - V_0: the potential funciton
    - Order: the order of Taylor expansion at 0

Return:
    [f_0, tV_0, tf_0]:

Note:
    prefix t means Taylor expansions

"""
function getFunctions(V_0, Order)
    f_0(x) = -ForwardDiff.derivative(V_0, x)

    # Taylor expansion
    t = Taylor1(Float64, Order)
    tV_0 = V_0(t)
    tf_0 = -derivative(tV_0)
    
    return [f_0, tV_0, tf_0]
end

f_0, tV_0, tf_0 = getFunctions(V_0, Order)

# Plot the Taylor expansion to make sure it looks fine

x = -4:0.2:4
y1 = V_0.(x)
y2 = tV_0.(x)
y3 = f_0.(x)
y4 = tf_0.(x)

# p1 = plot(x, y1, ylims)

plot(x, y1, ylims=(-2, 5), label="V_0")
p1 = plot!(x, y2, ylims=(-2, 5), label="taylor approx V_0")
plot(x, y3, ylims=(-2, 5), label="f_0")
p2 = plot!(x, y4, ylims=(-2,5), label="taylor approx f_0")
plot(p1, p2, layout = (1, 2))

# Scalar function (m=1)

In [ ]:
# Extended dimension
N = 2

"""
The embedding function
Argument:
    - X: value to evaluate (N, 1)
    - a: the Taylor coefficients of the function
    - commutative (default true): if the embedding function is commutative
Return:
    - The value of the embedding (N, 1)
"""
function CommF(X, a, commutative=true)
    
    N = length(X)
    M = length(a) - 1
    result = zeros(N, N)
    
    Ω₁ = fill(1, (N, N)) / N
    
    for i=0:M
        if commutative == 1
            result += a[i+1] * Ω₁ * diagm(X)^i
        else
            result += a[i+1] * (Ω₁ * diagm(X))^i
        end
    end
    return result * fill(1, (N, 1)) # b is set to be 1 here, but it can be more generalized
end

X = [1; 2]
const a = tf_0.coeffs
const M = length(a) - 1


display(CommF(X, a))
display(CommF(X, a, false))


$$
\frac{dX}{dt} = \sum_{i=0}^N \Omega_1 a_i X^i \vec{1} - \alpha (I - \Omega_1) \vec{1}
$$

In [ ]:
#=
"""
The derivative function for the dynamics
Argument:
    - dX: the placeholder to receive output
    - X: the value to evaluate
    - p: the parameter list
        - p[1]: the α in the argument
        - p[2]: whether to use commutative embedding function
        - p[3]: the number of a (i.e. N in the formula)
        - p[4..]: the list of a
Return:
    Inplace output in dX

"""
function f!(dX, X, p, t=0)
    N = length(X)
    Ω₁ = fill(1, (N, N)) / N
    
    α = p[1] # First parameter for α
    comm = p[2] # Second parameter for commutative function
    len_a = Integer(p[3])
    a = @view p[4:4+len_a - 1]
    
    result = CommF(X, a, comm)
    result -= α * (I - Ω₁) * X

    dX .= result
    dX
end


function direct_f(X, p, t=0)
    N = length(X)
    dX = zeros(Float64, N)
    f!(dX, X, p, t)
end
=#

"""
Return the average gradient, given the gradient function
TODO: also the original function?
"""
function ExtendedF!(dX, X, f, alpha)
    N = length(X)
    Omega1 = fill(1, (N, N)) / N
    first_term = mean(f.(X))
    second_term = -alpha * (I - Omega1) * X
    
    if !isnothing(dX)
        dX .= first_term .+ second_term
    end
    first_term .+ second_term
end
function ExtendedF(X, f, alpha)
    ExtendedF!(nothing, X, f, alpha)
end

function GenerateF(f, alpha)
   (dX, X, p=0, t=0) -> ExtendedF!(dX, X, f, alpha)
end


dX = [0., 0.]

# p = vcat([1, true, length(a)], a)


# println(f!(dX, [1, 2], p))
println(dX)
# println(direct_f([1, 2], p))

println(ExtendedF([1,2], f_0, 1))
println(GenerateF(f_0, 1)(nothing, [1,2]))

## One dynamic

In [ ]:
printMsg("One particle dynamic")

#="""
Get the solution of the ODE problem defined by f!
Argument:
    - X₀: the initial value
    - tspan: the time range for the problem
    - p: the parameter to pass in the problem
Return:
    - The solution

"""
function getSolution(X₀, f, tspan, p; ts=0.1, verbose=true)
    prob = ODEProblem(f!, X₀, tspan, p)
    sol = solve(prob, saveat=ts, verbose=verbose)
    return sol
end

sol = getSolution(X₀, tspan, p, verbose=false)

=#

X₀ = [3; 2]
tspan = (0, 10)

grad_f = GenerateF(f_0, 1)

prob = ODEProblem(grad_f, [3; 2], tspan, 0)
sol = solve(prob)


x = [mean(vec) for vec in sol.u]

println(x[end])
plot(x, label="x")



### Slope field

In [ ]:
meshgrid(x, y) = (repeat(x, outer=length(y)), repeat(y, inner=length(x)))
x, y = meshgrid(-4:0.2:4, -4:0.2:4)

u = zeros(Float64, length(x))
v = zeros(Float64, length(y))

tmp = map((xi, yi) -> grad_f(nothing, [xi; yi]), x, y)

u = [i[1] for i in tmp]
v = [i[2] for i in tmp]

scale = 0.002
u .= scale * u
v .= scale * v
quiver(x, y, quiver=(u, v))
slope_field = plot!(size=(400,400))

### Initial points

In [ ]:
plot()

x, y = meshgrid(-4:0.1:4, -4:0.1:4)
grid = [[a, b] for (a, b) in zip(x, y)]

global_count, local_count = 0, 0

for i in 1:1000
    X0 =  -4. .+ 8. .* rand(2)
    
    prob = ODEProblem(grad_f, X0, tspan)
    sol = solve(prob)
    # sol = getSolution(X0, (0, 10), p)
    
    global global_count, local_count

    # Extract x and y values
    x_values = [point[1] for point in sol.u]
    y_values = [point[2] for point in sol.u]

    # Create scatter plot
    if y_values[end] < 0
        color = :blue
        global_count += 1
    else
        color = :red
        local_count += 1
    end
    
    plot!(x_values, y_values, title="Results of different initial points", 
        xlabel="x", ylabel="y", legend=false, linecolor = color)
end

@printf("Probability of global convergence: %.2f%%\n", global_count / (global_count + local_count) * 100)
# println()

plot!(size=(800,800))


## Two particles dynamics

In [ ]:
printMsg("Two particle dynamic")

In [ ]:
X0 = [4;-1.5]
prob = ODEProblem(grad_f, X0, tspan)
sol = solve(prob)
# sol = getSolution(X0, (0, 10), p)
x1 = [i[1] for i in sol.u]
x2 = [i[2] for i in sol.u]

y1 = V_0.(x1)
y2 = V_0.(x2)
println()
# plot(slope_field)
# scatter!(x1, x2, label='X')

In [ ]:
anim = @animate for i in eachindex(x1)
    plot(V_0, -2, 4, label = "Potential")
    scatter!([x1[i]], [y1[i]], label = "x1")
    p1 = scatter!([x2[i]], [y2[i]], label = "x2")
    
    plot(slope_field)
    p2 = scatter!([x1[i]], [x2[i]], label="X")
    
    plot(p1, p2, layout=(1,2), size = (1200, 600))
end fps = 2
gif(anim, "dynamic.gif", fps=2)

In [ ]:
anim = @animate for i in eachindex(x1)
    plot(V_0, -2, 4, label = "Potential")
    scatter!([x1[i]], [y1[i]], label = "x1")
    scatter!([x2[i]], [y2[i]], label = "x2")
end fps = 2
gif(anim, "potential.gif", fps=2)

## N particle dynamics

In [ ]:
printMsg("N particle dynamic")

In [ ]:
V_0(x) = x^8 - 8x^6 + 19x^4 - 12x^2 + 2x + 2
f_0 = x -> -ForwardDiff.derivative(V_0, x)
grad_f = GenerateF(f_0, 1)

#=
f_0, tV_0, tf_0 = getFunctions(V_0, 8)
p = vcat([1, true, length(tf_0.coeffs)], tf_0.coeffs)
=#

In [ ]:
plot(V_0, -2.3, 2.3, label="V")

In [ ]:
N = 10
Ω₁ = fill(1, (N, N)) / N

# X0 = [4;-1.5; 1.2; 1.3]
X0 = (rand(N) .- 0.5) .* 2 .* 2
# X0 = [2.006145848242574, -1.2094066791560183, 2.412284620250457, 1.9682204891942996]

prob = ODEProblem(grad_f, X0, tspan)
sol = solve(prob)

# sol = getSolution(X0, (0, 10), p)

println(X0)

In [ ]:
anim = @animate for i in eachindex(sol.u)
    lower = min(minimum(sol.u[i]) - 0.5, -2.3)
    upper = max(maximum(sol.u[i]) + 0.5, 2.3)
    lower, upper = -2.3, 2.3

    plot(V_0, lower, upper, label = "Potential")
    for j in eachindex(sol.u[1])
        x = sol.u[i][j]
        y = V_0(x)
        scatter!([x], [y], label = "x$j")
    end
    # scatter!([x1[i]], [y1[i]], label = "x1")
    # scatter!([x2[i]], [y2[i]], label = "x2")
    # scatter!([x3[i]], [y3[i]], label = "x3")
end fps = 30

@printf("Converge to %.2f", mean(sol.u[end]))

gif(anim, "dynamic.gif")

### Experiments for different N

In [ ]:
struct GridStruct
    lower::Float64
    upper::Float64
    grid_size::Float64
    dimension::Int64
end

function Base.iterate(g::GridStruct)
    res = fill(g.lower, g.dimension)
    state = fill(1, g.dimension)
    
    return res, state
end

function Base.iterate(g::GridStruct, state::Vector{Int64})
    one_grid = collect(g.lower:g.grid_size:g.upper)
    max_index = length(one_grid)
    
    index = length(state)
    
    while index > 0
        state[index] += 1
        if state[index] > max_index
            if index > 1
                reset = state[index-1] + 1
                for j in index:length(state)
                    state[j] = reset
                end
                # state[index] = state[index - 1] + 1
            end
            index -= 1
        else
            res = zeros(g.dimension)
            # println(state)
            # println(one_grid)
            for i in eachindex(res)
                res[i] = one_grid[state[i]]
            end
            return res, state
        end
    end
    
    return nothing
end

function Base.length(g::GridStruct)
    count = 0
    for x in g
        count += 1
    end
    count
end
    
grid = GridStruct(-2, 2, 0.2, 2)
length(grid)

In [ ]:

function grid_experiment(N, X_list_iterator, optimum, tol=0.1; verbose=true)        
    # Create a progress bar
    verbose && (progress_bar = Progress(length(X_list_iterator), false))
    
    count = Atomic{Int64}(0)
    @threads for X0 in X_list_iterator
        
        # println(X0)
        
        prob = ODEProblem(grad_f, X0, tspan)
        sol = solve(prob, Rosenbrock23(), verbose=true)
        
        if abs(mean(sol.u[end]) - optimum) < tol
            atomic_add!(count,1)
        end
        verbose && next!(progress_bar)
    end
    return count[] / length(X_list_iterator)
end

In [ ]:
printMsg("Running grid experiment")

N_results = Float64[]

if IN_SCRIPT
    upper = 8
else
    upper = 2
end

for N=1:upper
    X_list_iterator = collect(GridStruct(-2, 2, 0.2, N))
    println("The total number of sample: ", length(X_list_iterator))
    push!(N_results, grid_experiment(N, X_list_iterator, -1.89))
    println("N_results: ", N_results)
    flush(stdout)
end

For the potential function
$$
V(x) = x^8 - 8x^6 + 19x^4 - 12x^2 + 2x + 2
$$

The results (ran on HPC) are 
| Number of Particles (N) | Global Convergence Probability | Grid Size |
|------------------------|-------------------------------|-----------|
| 1                      | 0.19047619047619047          | 0.2       |
| 2                      | 0.30385487528344673          | 0.2       |
| 3                      | 0.3911024727351258           | 0.2       |
| 4                      | 0.3213990055583836           | 0.2       |
| 5                      | 0.35958795661001797          | 0.4       |
| 6                      | 0.3716778120713306           | 0.8       |


This is from the new sampler (non-repetitive)

0.19047619047619047, 0.2987012987012987, 0.37662337662337664, 0.3035949557688688, 0.313382269904009, 0.3257655388090171, 0.32579079535601274, 0.3352065004238917

| Number of Particles (N) | Global Convergence Probability | Grid Size |
|------------------------|-------------------------------|-----------|
| 1                      | 0.19047619047619047          | 0.2       |
| 2                      | 0.2987012987012987           | 0.2       |
| 3                      | 0.37662337662337664          | 0.2       |
| 4                      | 0.3035949557688688           | 0.2       |
| 5                      | 0.313382269904009            | 0.2       |
| 6                      | 0.3257655388090171           | 0.2       |
| 7                      | 0.32579079535601274          | 0.2       |
| 8                      | 0.3352065004238917           | 0.2       |


In [ ]:
# Define the number of particles and corresponding convergence probabilities
num_particles = [1, 2, 3, 4, 5, 6, 7, 8]
convergence_probabilities = [0.19047619047619047, 0.2987012987012987, 0.37662337662337664, 0.3035949557688688, 
                            0.313382269904009, 0.3257655388090171, 0.32579079535601274, 0.3352065004238917]

# Create the plot
plot(num_particles, convergence_probabilities, marker = :circle, markersize = 6,
     xlabel = "Number of Particles (N)", ylabel = "Probability",
     title = "Probability of Convergence to Global Minimum",
     legend = false)

### Uniform sampling

In [ ]:
struct UniformSampler
    dimension::Int64
    one_grid::StepRangeLen{Float64}
    size::Int64
end

function Base.iterate(g::UniformSampler)
    return rand(g.one_grid, g.dimension), 1
end

function Base.iterate(g::UniformSampler, state::Int64)
    if state >= g.size
        return nothing
    else
        return rand(g.one_grid, g.dimension), state + 1
    end
end

function Base.length(g::UniformSampler)
    count = 0
    for x in g
        count += 1
    end
    count
end
    
unif = UniformSampler(4, -2:0.2:2, 3)
println(length(unif))
for i in unif
    println(i)
end

In [ ]:
printMsg("Running uniformly sampled experiment")

N_results = Float64[]

if IN_SCRIPT
    upper = 20
    sample_size = 1e5
else
    upper = 2
    sample_size = 1e4
end

one_grid = -2:0.2:2

println("The total number of sample: ", sample_size)

for N=1:upper
    unif_sampler = collect(UniformSampler(N, one_grid, sample_size))
    push!(N_results, grid_experiment(N, unif_sampler, -1.89, verbose=false))
    flush(stdout)
    print(".")
end
println()
println("N_results: ", N_results)

Result is 

The sample size is 1e4

0.186, 0.3015, 0.384, 0.3244, 0.3376, 0.3544, 0.3582, 0.3747, 0.3791, 0.3955, 0.4093, 0.42, 0.4382, 0.443, 0.4569, 0.4667, 0.4675, 0.4843, 0.5066, 0.5099

Sample size 1e5

0.18913, 0.30496, 0.38978, 0.3198, 0.33514, 0.35304, 0.36242, 0.37046, 0.3859, 0.40086, 0.41116, 0.4261, 0.43462, 0.44551, 0.45521, 0.46302, 0.47914, 0.48901, 0.498, 0.50454

In [ ]:
# Define the number of particles and corresponding convergence probabilities
convergence_probabilities = [0.18913, 0.30496, 0.38978, 0.3198, 0.33514, 0.35304, 0.36242, 0.37046, 0.3859, 0.40086, 0.41116, 0.4261, 0.43462, 0.44551, 0.45521, 0.46302, 0.47914, 0.48901, 0.498, 0.50454]
num_particles = collect(1:length(convergence_probabilities))

scaled = [1 - (1-convergence_probabilities[1])^i for i in num_particles]

# Create the plot
plot(num_particles, convergence_probabilities, marker = :circle, markersize = 6,
     xlabel = "Number of Particles (N)", ylabel = "Probability of Convergence to Global Minimum",
     title = "Uniformly sampled on the grid with 1e4 samples",
     legend = false)

plot!(num_particles, scaled, marker = :circle, markersize = 6)

## Vector function (m = 2)

We will focus on m=2 first

We will do mixture of Gaussians and classical XY model

Let's first plot some functions.



### Gaussian

In [ ]:
# Expect Sigma to be diagonal
Gaussian(X, mu, Sigma) = -1 / sqrt((2*pi)^length(X) * det(Sigma)) * exp(-1/2 * (X-mu)' * inv(Sigma) * (X-mu))
Gaussian_grad(X, mu, Sigma) = -Gaussian(X, mu, Sigma) * inv(Sigma) * (X-mu)

In [ ]:
g1 = x -> Gaussian(x, 0, 1)
g2 = x -> Gaussian_grad(x, 0, 1)

grad_g = (u, p, t) -> -g2(u)

prob = ODEProblem(grad_g, 1, tspan)
sol = solve(prob)
# display(sol)

plot(g1)
scatter!(sol.u, g1.(sol.u))

In [ ]:
mu = [0,0]
Sigma = Diagonal([1,1])
g1 = X -> Gaussian(X, mu, Sigma)
# g22 = X -> ForwardDiff.gradient(g1, X)
g2 = X -> Gaussian_grad(X, mu, Sigma)
g3 = (x,y) -> g1([x,y])

grad_g = (U, p, t) -> -g2(U)

prob = ODEProblem(grad_g, [1, 1], (0,20))
sol = solve(prob)

surface(one_grid, one_grid, g3)

xs = [i[1] for i in sol.u]
ys = [i[2] for i in sol.u]

zs = g1.(sol.u)

scatter!(xs, ys, zs)

### Mixture of Gaussians

In [ ]:
N = 2
mu1 = [-2,0]
mu2 = [2,0]

Sigma1, Sigma2 = Diagonal([2,1]), Diagonal([1,1])

gm1 = X -> (Gaussian(X, mu1, Sigma1) + Gaussian(X, mu2, Sigma2))
gm2 = X -> (Gaussian_grad(X, mu1, Sigma1) + Gaussian_grad(X, mu2, Sigma2))
gm3 = (x,y) -> gm1([x,y])
gm4 = (x,y) -> gm2([x,y])

In [ ]:
one_grid_large = -8:0.1:8
surface(one_grid_large, one_grid_large, gm3)

In [ ]:
grad_g = (U, p, t) -> -gm2(U)

prob = ODEProblem(grad_g, [1, 1], (0,40))
sol = solve(prob)

surface(one_grid_large, one_grid_large, gm3, legend=false)

xs = [i[1] for i in sol.u]
ys = [i[2] for i in sol.u]

zs = gm1.(sol.u)

p1 = deepcopy(scatter!(xs, ys, zs))
p2 = plot!(camera=(0,90))

plot(p1, p2, layout=(1,2), size = (800,400))

In [ ]:
function ExtendedF!(dX, X, f, alpha)
    N = length(X)
    Omega1 = fill(1, (N, N)) / N
    first_term = mean(f.(X))
    second_term = -alpha * (I - Omega1) * X
    
    if !isnothing(dX)
        dX .= first_term .+ second_term
    end
    first_term .+ second_term
end
function ExtendedF(X, f, alpha)
    ExtendedF!(nothing, X, f, alpha)
end

function GenerateF(f, alpha)
   (dX, X, p=0, t=0) -> ExtendedF!(dX, X, f, alpha)
end


In [ ]:
# Input X should be a matrix of shape N * m, similarly for dX
function VectorExtendedF!(dX, X, F, alpha)
    N, m = size(X)
    Omega1 = fill(1, (N, N)) / N
    first_term = mean(F.([X[i,:] for i in 1:N])) # TODO: need to optimize
    first_term = reshape(first_term, 1, length(first_term))
    second_term = -alpha * (I - Omega1) * X
    # println("First term size $(typeof(first_term))\n Second term size $(typeof(second_term))")
    if !isnothing(dX)
        dX .= first_term .+ second_term
    end
    first_term .+ second_term
end

function VectorExtendedF(X, F, alpha)
    VectorExtendedF!(nothing, X, F, alpha)
end

function VectorGenerateF(F, alpha)
   (dX, X, p=0, t=0) -> VectorExtendedF!(dX, X, F, alpha)
end

VectorExtendedF!(nothing, [2 0; 2 0; 2 0], X -> -gm2(X), 0.1)



In [ ]:
grad_g = VectorGenerateF(X -> -gm2(X), 0.1)
X0 = [-2.3 1; 2 -1; 2.5 1.5]
# X0 = [-2.3 1; -2 -1; 2.5 1.5]
# X0 = [2 -1;]
prob = ODEProblem(grad_g, X0, (0,100))
sol = solve(prob) # vector of matrix. T * [N * m]
println()

In [ ]:
function matVec2Arr(mv)
    a = zeros(size(mv[1])..., length(mv))
    for i in 1:length(mv)
        a[:,:,i] = mv[i]
    end
    return a
end

sols = matVec2Arr(sol.u)
sols = permutedims(sols, (1, 3, 2))

_, T, _ = size(sols) # m, Timestamp, Dimension of the original problem

In [ ]:
# Static version
surface(one_grid_large, one_grid_large, gm3, legend=false, size=(800,800))

N, m = size(X0)

for ind in 1:size(sols)[1]
    xs = [sols[ind,t,1] for t in 1:size(sols)[2]]
    ys = [sols[ind,t,2] for t in 1:size(sols)[2]]
    # println(ys)
    zs = gm3.(xs, ys)
    scatter!(xs,ys,zs, label="particle$ind", markersize=3)
end
plot!()
# zs = gm1.(sol.u)

p1 = deepcopy(plot!())
p2 = plot!(camera=(0,90))

plot(p1, p2, layout=(1,2), size = (800,400))

In [ ]:
# Gif
N, m = size(X0)
anim = @animate for t in 1:T
    surface(one_grid_large, one_grid_large, gm3, legend=false)

    for ind in 1:size(sols)[1]
        xs = [sols[ind,t,1]]
        ys = [sols[ind,t,2]]
        # println(ys)
        zs = gm3.(xs, ys)
        scatter!(xs,ys,zs, label="particle$ind", markersize=3)
    end
    plot!()
    # zs = gm1.(sol.u)

    p1 = deepcopy(plot!())
    p2 = plot!(camera=(0,90.1))

    plot(p1, p2, layout=(1,2), size = (800,400))
end

gif(anim, "Mixture_of_gaussian.gif", fps=5)

### Experiments

In [ ]:
typeof((2,3))

In [ ]:
struct MultipleDimUniformSampler
    shape::Tuple{Vararg{Int64}}
    one_grid::StepRangeLen{Float64}
    size::Int64
end

function Base.iterate(g::MultipleDimUniformSampler)
    return rand(g.one_grid, g.shape), 1
end

function Base.iterate(g::MultipleDimUniformSampler, state::Int64)
    if state >= g.size
        return nothing
    else
        return rand(g.one_grid, g.shape), state + 1
    end
end

function Base.length(g::MultipleDimUniformSampler)
    count = 0
    for x in g
        count += 1
    end
    count
end
    
unif = MultipleDimUniformSampler((4,3), -2:0.2:2, 3)
println(length(unif))
for i in unif
    println(i)
end

In [ ]:
# N is not used
function multi_grid_experiment(X_list_iterator, optimum, tol=0.1; verbose=true)        
    # Create a progress bar
    verbose && (progress_bar = Progress(length(X_list_iterator), false))
    
    count = Atomic{Int64}(0)
    @threads for X0 in X_list_iterator
        
        prob = ODEProblem(grad_g, X0, (0,100))
        sol = solve(prob, Rosenbrock23(), verbose=true)
        
        if norm(vec(mean(sol.u[end], dims=1)) - optimum) < tol
            atomic_add!(count,1)
        end
        verbose && next!(progress_bar)
    end
    return count[] / length(X_list_iterator)
end

In [ ]:
printMsg("Running uniformly sampled experiment")

N_results = Float64[]

if IN_SCRIPT
    upper = 20
    sample_size = 1e5
else
    upper = 8
    sample_size = 1e4
end


println("The total number of sample: ", sample_size)

m = 2

for N=1:upper
    unif_sampler = collect(MultipleDimUniformSampler((N, m), one_grid, sample_size))
    push!(N_results, multi_grid_experiment(unif_sampler, mu2, verbose=false))
    flush(stdout)
    print(".")
end
println()
println("N_results: ", N_results)

In [ ]:
# Define the number of particles and corresponding convergence probabilities
convergence_probabilities = N_results
num_particles = collect(1:length(convergence_probabilities))

scaled = [1 - (1-convergence_probabilities[1])^i for i in num_particles]

# Create the plot
plot(num_particles, convergence_probabilities, marker = :circle, markersize = 6,
     xlabel = "Number of Particles (N)", ylabel = "Probability of Convergence to Global Minimum",
     title = "Uniformly sampled on the grid with 1e4 samples",
     legend = false)

# plot!(num_particles, scaled, marker = :circle, markersize = 6)